In [ ]:
import matplotlib.pyplot as plt
from hydromt_sfincs import SfincsModel, utils
import xarray as xr
import os
import numpy as np
from os.path import join
import matplotlib.pyplot as plt
import hydromt 
from hydromt_sfincs import SfincsModel
from matplotlib import animation
from hydromt.config import configread
from hydromt.log import setuplog


## Get SFINCS model

In [ ]:
#sfincs_root_base = r"c:\git\sfincs_alblasserwaard\sfincs_discharge_base"  # (relative) path to sfincs root

sfincs_root = r"c:\USVI\2_Modelsetup\domain_0021"
#scenario_name = f"WL_{waterlevel}_breach_{breach}_depth_adjust_{depth_adjust}"

mod = SfincsModel(sfincs_root, mode="r")




## Plotting maximum waterlevel

In [ ]:
zs = mod.results["zs"]
zsmax = zs.max(dim='time')

bed_level =  mod.results["zb"].copy()
water_level_max = zsmax - bed_level

water_level_max

# hmax = utils.downscale_floodmap(
#     zsmax=water_level_max,
#     dep=dep, #Use subgrid
#     hmin=hmin,
#     # gdf_mask=gdf_osm,
#     # floodmap_fn=join(sfincs_root, "floodmap.tif") # uncomment to save to <mod.root>/floodmap.tif
# )


fig, ax = mod.plot_basemap(
    fn_out=None,
    variable="", # no variable to plot, only basemap
    plot_bounds=False, 
    bmap="sat",
    figsize=(11, 7),
    zoomlevel = 12
)


cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
cax_fld = water_level_max.plot(
    x="xc", y="yc",
    ax=ax,
    vmin=0.0, vmax=2.0,
    cmap=plt.cm.viridis,
    cbar_kwargs=cbar_kwargs,
)

ax.set_title(f"SFINCS maximum water depth")
#plt.savefig(join(scenario_dir, 'hmax.png'), dpi=225, bbox_inches="tight")
plt.show()


## Amimation

In [ ]:
print("Make animation")

hmin = 0.05
da_h = mod.results["zs"].copy() - mod.results["zb"].copy()
da_h = da_h.where(da_h > hmin).drop("spatial_ref")
da_h.attrs.update(long_name="flood depth", unit="m")

step = 1  # one frame every <step> dtout
cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}


def update_plot(i, da_h, cax_h):
    da_hi = da_h.isel(time=i)
    t = da_hi.time.dt.strftime("%d-%B-%Y %H:%M:%S").item()
    print( f"Animation t= {t}")
    ax.set_title(f"SFINCS water depth {t}")
    cax_h.set_array(da_hi.values.ravel())


fig, ax = mod.plot_basemap(
    fn_out=None, variable="", bmap="sat", plot_bounds=False, figsize=(11, 7)
)
cax_h = da_h.isel(time=0).plot(
    x="xc", y="yc",
    ax=ax,
    vmin=0, vmax=2,
    cmap=plt.cm.viridis,
    cbar_kwargs=cbar_kwargs
)
plt.close()  # to prevent double plot

ani = animation.FuncAnimation(
    fig,
    update_plot,
    frames=np.arange(0, da_h.time.size, step),
    interval=250,  # ms between frames
    fargs=(
        da_h,
        cax_h,
    ),
)

# to save to mp4
ani.save(join(sfincs_root , 'sfincs_h.mp4'), fps=4, dpi=200)


## To Geotiff

In [ ]:
water_level_max.raster.to_raster(
                                join(sfincs_root, "gis", "hsmax.tif"),
                                driver="GTiff",
                                compress="deflate",
                            )

#Let op hsmax toevoegn in hydroymsfincs broncode (sfincs.py)

## Downscale floodmap


depfile = join(sfincs_root, "subgrid", "dep_subgrid.tif")
da_dep = mod.data_catalog.get_rasterdataset(depfile)

hmin = 0.01

hmax = utils.downscale_floodmap(
    zsmax=zsmax,
    dep=da_dep, #Use subgrid
    hmin=hmin,
    # gdf_mask=gdf_osm,
    floodmap_fn=join(sfincs_root, "subgrid", "floodmap.tif") # uncomment to save to <mod.root>/floodmap.tif
)